In [1]:
import pandas as pd
import urllib
import time
import json

In [113]:
studentBehavior = pd.read_json("studentBehaviorInfoOver40Class_1213.json", orient="index")
csv_file = pd.read_csv("clusterVideoAllDimensions.csv",header=None)

In [34]:
# print(studentBehavior[0])
# print studentBehavior[0]["memberId"]
# print (studentBehavior[0]["chosenVideo"])
# print (studentBehavior[0]["vocabularyList"])
print(video_to_cluster)

## Desired json structure
# [
#     {
#         "memberId": 12345,
#         "videoSequence":
#             [
#                 3980,
#                 7680,
#                 1235
#             ],
#         "skippingRatioSequence":
#             [
#                 0,
#                 0,
#                 0.3
#             ],
#         "scoreSequence":
#             [
#                 80,
#                 70,
#                 75
#             ],
#         "dictionarySequence":
#             [
#                 0,
#                 0,
#                 0
#             ],
#         "skippingIndex": 10,
#         "clusterSequence":
#             [
#                 "C1",
#                 "C1",
#                 "C2"
#             ]
        
#     }
# ]



13354


In [114]:
# Generate Users Profile File

# This method return a mapping of video(key) 
# and and array of scores(value)
def parse_video_scores(scores_object):
    result = {}
    for score in scores_object:
        if score['postId'] not in result:
            result[score['postId']] = []
        result[score['postId']].append(score['score'])
    return result
            
def parse_video_words(words_object):
    result = {}
    for word in words_object:
        if word['postId'] not in result:
            result[word['postId']] = []
        result[word['postId']].append(word['word'])
    return result

def parse_csv_file(csv_file):
    result = {}
    for index in range(1,len(csv_file[0])):
        result[csv_file[0][index]] = int(csv_file[1][index])
    return result

def get_cluster_cardinality(videos_cluster):
    result = {}
    for video in videos_cluster:
        if videos_cluster[video] in result:
            # Increase
            result[videos_cluster[video]] += 1
        else:
            result[videos_cluster[video]] = 1
    return result
    

def calc_skipping_ratio(scores):
    countSkipped = 0
    for score in scores:
        if score < 0:
            countSkipped += 1
    return round(countSkipped*1.0/len(scores)*1.0,2)

def calc_avg_score(scores):
    countSkipped = 0
    sumScores = 0
    for score in scores:
        if score < 0:
            countSkipped += 1
        else:
            sumScores += score
                
    if countSkipped > 0:
        return 0
    else:
        return round(sumScores*1.0/(len(scores)*1.0-countSkipped*1.0),2)
    
lenStudentBehavior = len(pd.read_json("studentBehaviorInfoOver40Class_1213.json"))
usersProfilesJsonWithIndexes = {}
video_to_cluster = parse_csv_file(csv_file)
cardinality = get_cluster_cardinality(video_to_cluster)
print(cardinality)
for index in range(lenStudentBehavior):
    memberId = studentBehavior[index]["memberId"]
    if memberId not in usersProfilesJsonWithIndexes:
        studentProfile                          = {}
        studentProfile["memberId"]              = memberId
        studentProfile["videoSequence"]         = []
        studentProfile["skippingRatioSequence"] = []
        studentProfile["avgScoreSequence"]         = []
        studentProfile["dictionarySequence"]    = []
        studentProfile["skippingIndex"]         = -1
        studentProfile["clusterSequence"]       = []
        studentProfile["scores"]                = []
        studentProfile["interaction"]           = {}
        usersProfilesJsonWithIndexes[memberId] = studentProfile
    
    # Addid values for every key
    # Video Sequence
    usersProfilesJsonWithIndexes[memberId]["videoSequence"].extend(studentBehavior[index]["chosenVideo"])
    
    
    videos_scores = parse_video_scores(studentBehavior[index]["listenScore"])
    video_words = parse_video_words(studentBehavior[index]["vocabularyList"])
    
    
    # SkippingRationSequence
    for video in studentBehavior[index]["chosenVideo"]:
        scores = videos_scores[video]
        usersProfilesJsonWithIndexes[memberId]["scores"].append(scores)
        usersProfilesJsonWithIndexes[memberId]["skippingRatioSequence"].append(calc_skipping_ratio(scores))
        # Average Score Sequence
        avg = calc_avg_score(scores)
        usersProfilesJsonWithIndexes[memberId]["avgScoreSequence"].append(avg)
        
        # DictionarySequence
        if str(video) in video_words:
            words = video_words[str(video)]
            usersProfilesJsonWithIndexes[memberId]["dictionarySequence"].append(len(words))
        else:
            usersProfilesJsonWithIndexes[memberId]["dictionarySequence"].append(0)
            
        # ClusterSequece
        usersProfilesJsonWithIndexes[memberId]["clusterSequence"].append(video_to_cluster[str(video)])
            
        
    # skippingIndex
    skip_index = [ n for n,i in enumerate(usersProfilesJsonWithIndexes[memberId]["skippingRatioSequence"]) if i>0.0 ][0]
    usersProfilesJsonWithIndexes[memberId]["skippingIndex"] = skip_index
    # Add cluster turning point
    usersProfilesJsonWithIndexes[memberId]["tp"] = usersProfilesJsonWithIndexes[memberId]["clusterSequence"][skip_index]
    
    # Calculate Interaction
    # Sum scores by cluster
    scores_by_cluster = {}
    for c_index in range(len(usersProfilesJsonWithIndexes[memberId]["clusterSequence"])):
        cluster = usersProfilesJsonWithIndexes[memberId]["clusterSequence"][c_index]
        if cluster in scores_by_cluster:
            scores_by_cluster[cluster] += usersProfilesJsonWithIndexes[memberId]["avgScoreSequence"][c_index]
        else:
            scores_by_cluster[cluster] = usersProfilesJsonWithIndexes[memberId]["avgScoreSequence"][c_index]
    # Add cluster with no interaction
    for cluster in cardinality:
        if cluster not in scores_by_cluster:
            scores_by_cluster[cluster] = 0
    # Divide sum by cardinality of the cluster
    for cluster in scores_by_cluster:
        scores_by_cluster[cluster] = round((scores_by_cluster[cluster]*1.0) / (cardinality[cluster] * 1.0),2)
    
    usersProfilesJsonWithIndexes[memberId]["interaction"] = scores_by_cluster
    
    
        
print(usersProfilesJsonWithIndexes['50679'])
fileStudentProfile = open("studentProfile_all_dimensions.json", "w")
fileStudentProfile.write("[")
lenObjects = len(usersProfilesJsonWithIndexes)
it = 0
for key, value in usersProfilesJsonWithIndexes.items():
    json.dump(value,fileStudentProfile)
    if it != (lenObjects-1):
        fileStudentProfile.write(", ")
    it = it + 1
    


fileStudentProfile.write("]")
fileStudentProfile.close()

{0: 50, 1: 58, 2: 81, 3: 46}
{'avgScoreSequence': [92.0, 93.33, 90.14, 90.17, 93.86, 92.0, 0, 0], 'videoSequence': [3913, 5186, 4974, 4802, 3711, 5881, 7126, 5797], 'memberId': '50679', 'dictionarySequence': [26, 15, 73, 32, 33, 17, 19, 40], 'scores': [[93, 91, 92, 92, 92], [93, 94, 93], [90, 80, 77, 96, 86, 97, 90, 95, 95, 98, 91, 86, 95, 86], [85, 90, 90, 93, 90, 93], [92, 88, 99, 94, 98, 91, 95], [90, 94], [77, 79, -1], [-1, -1, -1, -1, -1]], 'skippingIndex': 6, 'clusterSequence': [3, 3, 3, 1, 3, 3, 2, 1], 'tp': 2, 'interaction': {0: 0.0, 1: 1.55, 2: 0.0, 3: 10.03}, 'skippingRatioSequence': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.33, 1.0]}


# Notes

1. We have three different skipRatio behavior
    User that goes from 0 to 1
    Users that goes from 0 to [a range gratter than 0] and stop
    Users that goes from 0 to [a range gratter than 0] and then skippRation 1

2. The skipIndex shows the index where the skipRation is gratter than 0
    


In [112]:
# Generate csv file for classifier
interaction = open("users_interaction_all_dimensions.csv", "w")
header = ""
for key in cardinality:
    header += "C" + str(key) 
    header += ","
header += "TP" # class label
header += "\n"
print(header)
interaction.write(header)
for user in usersProfilesJsonWithIndexes:
    s = ""
    for key in cardinality:
        s += str(usersProfilesJsonWithIndexes[user]["interaction"][key])+","
    s += "C"+str(usersProfilesJsonWithIndexes[user]["tp"])
    s += "\n"
    print(s)
    interaction.write(s)
interaction.close()

C0,C1,C2,C3,TP

2.51,16.76,4.51,5.4,C1

1.34,0.0,3.06,10.63,C3

2.94,0.0,2.28,4.25,C2

2.27,5.54,0.0,8.42,C3

0.0,0.0,1.65,24.41,C2

8.27,50.84,3.18,14.39,C0

0.0,0.0,0.89,4.74,C2

2.93,27.99,3.31,25.32,C1

2.93,9.32,0.0,2.92,C1

2.81,2.18,8.68,7.08,C1

1.27,5.95,1.47,4.0,C1

1.38,21.7,2.86,5.63,C1

5.61,0.0,12.13,11.12,C0

0.0,0.0,1.35,22.28,C3

1.37,0.0,1.51,10.8,C3

1.31,1.67,4.16,6.78,C1

1.44,0.0,4.76,12.5,C2

2.92,4.51,3.23,4.3,C2

11.04,0.0,6.4,12.96,C3

1.49,0.0,6.63,35.85,C3

1.66,0.0,4.53,13.78,C1

0.0,0.0,0.0,6.66,C0

4.33,6.75,11.07,36.56,C1

0.0,0.0,0.0,9.76,C0

0.0,0.0,0.0,17.7,C3

5.84,0.0,6.47,19.13,C2

0.84,0.0,0.0,7.75,C0

2.76,0.0,4.91,12.3,C0

1.3,3.86,2.4,5.14,C0

2.58,0.0,6.04,17.53,C2

1.38,13.07,1.52,4.3,C1

1.3,0.0,1.51,8.28,C3

0.0,11.51,7.43,4.05,C1

4.34,27.68,0.0,4.45,C1

0.0,0.0,5.71,29.43,C3

1.41,0.0,1.52,7.22,C1

2.48,0.0,1.22,6.29,C2

0.0,0.0,0.0,12.73,C3

0.0,0.0,1.66,11.75,C2

5.74,13.16,9.22,4.41,C1

2.8,2.25,4.72,2.92,C2

0.0,2.07,1.4,7.76,C1

4.87

In [101]:
# Generate secuence of videos by user
interaction = open("users_video_sequence.csv", "w")
for user in usersProfilesJsonWithIndexes:
    s = (','.join(str(x) for x in usersProfilesJsonWithIndexes[user]["videoSequence"][:usersProfilesJsonWithIndexes[user]["skippingIndex"]+1]))
    s += "\n"
    interaction.write(s)
interaction.close()

# Generate secuence of videos by cluster
interaction = open("users_video_sequence_by_cluster.csv", "w")
for user in usersProfilesJsonWithIndexes:
    s = (','.join(str(x) for x in usersProfilesJsonWithIndexes[user]["clusterSequence"][:usersProfilesJsonWithIndexes[user]["skippingIndex"]+1]))
    s += "\n"
    interaction.write(s)
interaction.close()

# TP videos
interaction = open("tp_videos.csv", "w")
array = []
for user in usersProfilesJsonWithIndexes:
    index = usersProfilesJsonWithIndexes[user]["skippingIndex"]
    array.append(usersProfilesJsonWithIndexes[user]["videoSequence"][index])
result = list(set(array))
for v in result:
    interaction.write(str(v) + "\n")
interaction.close()

# TP clusters
interaction = open("tp_clusters.csv", "w")
array = []
for user in usersProfilesJsonWithIndexes:
    index = usersProfilesJsonWithIndexes[user]["skippingIndex"]
    array.append(usersProfilesJsonWithIndexes[user]["clusterSequence"][index])
result = list(set(array))
for v in result:
    interaction.write(str(v) + "\n")
interaction.close()